In [1]:
from neo4j import GraphDatabase, Query, Record
from neo4j.exceptions import ServiceUnavailable, SessionError

In [16]:
uri = "bolt://83.229.84.12:7687"  
username = "tumaiReadonly"
password = "MAKEATHON2024"  
database = "graph2.db"
AUTH = (username, password)

driver = GraphDatabase.driver(uri, auth=AUTH)

In [3]:
edge_types = None
with driver.session(database=database) as session:
	data = session.run("CALL db.relationshipTypes() YIELD relationshipType RETURN relationshipType;")
	edge_types = [r["relationshipType"] for r in data]
print(edge_types)

['HAS_PARENT', 'MAPS_TO', 'LOCATED_IN', 'TRANSCRIBED_INTO', 'HAS_SEQUENCE', 'BELONGS_TO_PROTEIN', 'TRANSLATED_INTO', 'ASSOCIATED_WITH', 'HAS_MODIFICATION', 'HAS_MODIFIED_SITE', 'IS_SUBSTRATE_OF', 'IS_SUBUNIT_OF', 'CURATED_INTERACTS_WITH', 'COMPILED_INTERACTS_WITH', 'ACTS_ON', 'IS_BIOMARKER_OF_DISEASE', 'IS_QCMARKER_IN_TISSUE', 'VARIANT_FOUND_IN_CHROMOSOME', 'VARIANT_FOUND_IN_GENE', 'VARIANT_FOUND_IN_PROTEIN', 'CURATED_AFFECTS_INTERACTION_WITH', 'VARIANT_IS_CLINICALLY_RELEVANT', 'DETECTED_IN_PATHOLOGY_SAMPLE', 'MENTIONED_IN_PUBLICATION', 'FOUND_IN_PROTEIN', 'HAS_STRUCTURE', 'VARIANT_FOUND_IN_GWAS', 'STUDIES_TRAIT', 'ANNOTATED_IN_PATHWAY', 'PUBLISHED_IN', 'HAS_ENROLLED', 'BELONGS_TO_SUBJECT', 'HAS_PHENOTYPE', 'HAS_DISEASE', 'HAS_PROTEIN', 'HAS_DAMAGE']


In [22]:
edges = []
# HAS_DISEASE
t = edge_types.index("HAS_DISEASE")
with driver.session(database=database) as session:
	data = session.run("MATCH (a:Biological_sample)-[r:HAS_DISEASE]->(b) RETURN id(a), id(b)")
	patients = list(set(r["id(a)"] for r in data))
	edges += [(r["id(a)"], t, r["id(b)"]) for r in data]

# HAS_PROTEIN
t = edge_types.index("HAS_PROTEIN")
with driver.session(database=database) as session:
	data = session.run("MATCH (a:Biological_sample)-[r:HAS_PROTEIN]->(b) RETURN id(a), id(b)")
	edges += [(r["id(a)"], t, r["id(b)"]) for r in data]

# HAS_PHENOTYPE
t = edge_types.index("HAS_PHENOTYPE")
with driver.session(database=database) as session:
	data = session.run("MATCH (a:Biological_sample)-[r:HAS_PHENOTYPE]->(b) RETURN id(a), id(b)")
	edges += [(r["id(a)"], t, r["id(b)"]) for r in data]

print(edges.__len__(), patients.__len__())

9915 68


In [5]:
import torch

In [12]:
edge_index = torch.tensor([(int(t[0]), int(t[2])) for t in edges], dtype=torch.int64).t()
edge_index.shape

torch.Size([2, 10054])

In [13]:
edge_type = torch.tensor([t[1] for t in edges], dtype=torch.int64)
edge_type.shape

torch.Size([10054])

In [14]:
edge_weight = torch.ones(edge_type.shape[0], dtype=torch.float32)
edge_weight.shape

torch.Size([10054])

In [18]:
with driver.session(database=database) as session:
	data = session.run("MATCH (p:Biological_sample)-[e:HAS_DISEASE]->(n:Disease {name: 'control'}) RETURN id(p)")
	healthy_patients = [r["id(p)"] for r in data]
healthy_patients.__len__()

1

In [20]:
import numpy as np

In [23]:
patients = np.array(patients)
healthy_patients = np.array(healthy_patients)
ill_label = np.isin(patients, healthy_patients, invert=True).astype(np.float32)
ill_label.shape

(68,)